# Model

In [1]:
# import statements

import os
import io
import pandas as pd
import numpy as np
import difflib
from datetime import datetime, timedelta
import statistics

import gspread
from df2gspread import df2gspread as d2g
from df2gspread import gspread2df as g2d
from oauth2client.service_account import ServiceAccountCredentials

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

from sklearn.linear_model import BayesianRidge

from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier

from sklearn import metrics
from scipy.stats import norm
from sklearn.preprocessing import OneHotEncoder

## Data Cleaning + Manipulation

In [2]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
json_file_name = os.getcwd() + "/sportsbetting-376321-f2ada03a7020.json"
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)
spreadsheet_key = '1qZfM3myJ9naCc_AT7U20Zjv5UPMEa20sSTVuG7HlNNc'

### Betting Data

In [3]:
#### Load Data

wks_name = "BettingPros"
betting_df = g2d.download(gfile=spreadsheet_key, wks_name=wks_name, col_names=True, row_names=True, credentials=credentials)
betting_df = betting_df.reset_index()
print(betting_df.shape)

(7301, 4)


In [4]:
#### Cleaning

# drop index column(s)
betting_df = betting_df.drop(columns=['index'])

# rename Name to Player
betting_df = betting_df.rename(columns={"Name": "Player"})

# convert Line to float
betting_df = betting_df.astype({'Line': 'float'})

# drop na values
rows_to_drop = []
for index,row in betting_df.iterrows():
    for col in betting_df.columns:
        if(pd.isna(betting_df[col][index])):
            rows_to_drop.append(index)
betting_df = betting_df.drop(rows_to_drop)

In [5]:
#### Summary

print(betting_df.shape)
print(betting_df.dtypes)
betting_df.head(3)

(7301, 3)
Player     object
Line      float64
Date       object
dtype: object


,Player,Line,Date
0,Jayson Tatum,22.5,2022-10-18
1,Stephen Curry,23.5,2022-10-18
2,De'Anthony Melton,7.5,2022-10-18


### Player Game Data

In [6]:
#### Load Data

wks_name = "PlayerData"
player_df = g2d.download(gfile=spreadsheet_key, wks_name=wks_name, col_names=True, row_names=True, credentials=credentials)
player_df = player_df.reset_index()

print(player_df.shape)

(16139, 38)


In [7]:
#### Cleaning

# convert Home_Away into Boolean (Home=TRUE)
player_df['Home'] = player_df['Home_Away'] != '@'

# convert Result into Win, Team_PTS, Opp_PTS
player_df['Win'] = [s.split()[0] == 'W' for s in player_df['Result']]
player_df['Team_PTS'] = [int(s.split()[1][:s.index("-")-2]) for s in player_df['Result']]
player_df['Opp_PTS'] = [int(s.split()[1][s.index("-")-1:]) for s in player_df['Result']]

# rename columns
player_df = player_df.rename(columns={"PTS": "Player_PTS", 
                                      "MP": "Player_MP",
                                      "FG": "Player_FGM",
                                      "FGA": "Player_FGA",
                                      "FG%": "Player_FG%",
                                      "2P": "Player_2PM",
                                      "2PA": "Player_2PA",
                                      "2P%": "Player_2P%",
                                      "3P": "Player_3PM",
                                      "3PA": "Player_3PA",
                                      "3P%": "Player_3P%",
                                      "FT": "Player_FTM",
                                      "FTA": "Player_FTA",
                                      "FT%": "Player_FT%",
                                      "TS%": "Player_TS%",
                                      "ORB": "Player_ORB",
                                      "DRB": "Player_DRB",
                                      "TRB": "Player_TRB",
                                      "AST": "Player_AST",
                                      "STL": "Player_STL",
                                      "BLK": "Player_BLK",
                                      "TOV": "Player_TOV",
                                      "PF": "Player_PF",
                                      "GmSc": "Player_GmSc",
                                      "BPM": "Player_BPM"})

# change numerical column types
numerical_columns = ['Player_PTS', 'Player_MP',
       'Player_FGM', 'Player_FGA', 'Player_FG%', 'Player_2PM', 'Player_2PA',
       'Player_2P%', 'Player_3PM', 'Player_3PA', 'Player_3P%', 'Player_FTM',
       'Player_FTA', 'Player_FT%', 'Player_TS%', 'Player_ORB', 'Player_DRB',
       'Player_TRB', 'Player_AST', 'Player_STL', 'Player_BLK', 'Player_TOV',
       'Player_PF', 'Player_GmSc', 'Player_BPM', 'Team_PTS',
       'Opp_PTS']
for col in numerical_columns:
    player_df = player_df.astype({col: 'float'})
    
# create primary position column
player_df["Player_Pos"] = [s[0] for s in player_df['Pos.']]

# drop columns
columns_to_drop = ['index', 'Rk', 'Player-additional', 'Home_Away', 'Result', 'Age', 'GS','PTS.1',
                  'Pos.', 'Win', 'Team_PTS', 'Opp_PTS']
player_df = player_df.drop(columns=columns_to_drop)

In [8]:
#### Manipulation - keep track of average stats coming into game

stats = ['Player_PTS', 'Player_MP', 'Player_FGM', 'Player_FGA', 'Player_FG%', 'Player_2PM', 'Player_2PA',
       'Player_2P%', 'Player_3PM', 'Player_3PA', 'Player_3P%', 'Player_FTM',
       'Player_FTA', 'Player_FT%', 'Player_TS%', 'Player_ORB', 'Player_DRB',
       'Player_TRB', 'Player_AST', 'Player_STL', 'Player_BLK', 'Player_TOV',
       'Player_PF', 'Player_GmSc', 'Player_BPM']

players = player_df['Player'].unique()

# sort rows from beginning to end of season
player_df = player_df.sort_values(by=['Date'], ignore_index=True)

# data structure to hold all data
player_dict = {player:{stat:[] for stat in stats} for player in players}

# keep column of player PPG standard deviation
player_ppg_sd = []

for index,row in player_df.iterrows():
    player = row['Player']
    ppg_sd = statistics.stdev(player_dict[player]['Player_PTS']) if len(player_dict[player]['Player_PTS']) > 1 else 0
    player_ppg_sd.append(ppg_sd) 
    for stat in stats:
        temp = row[stat]
        player_df.at[index,stat] = statistics.mean(player_dict[player][stat]) if len(player_dict[player][stat]) > 0 else 0
        if not pd.isna(temp): player_dict[player][stat].append(temp)

# add column for player PPG standard deviation
player_df['Player_PTSSTDEV'] = player_ppg_sd

In [9]:
#### Summary

print(player_df.shape)
print(player_df.dtypes)
player_df[-5:]

(16139, 32)
Player              object
Player_PTS         float64
Date                object
Team                object
Opp                 object
Player_MP          float64
Player_FGM         float64
Player_FGA         float64
Player_FG%         float64
Player_2PM         float64
Player_2PA         float64
Player_2P%         float64
Player_3PM         float64
Player_3PA         float64
Player_3P%         float64
Player_FTM         float64
Player_FTA         float64
Player_FT%         float64
Player_TS%         float64
Player_ORB         float64
Player_DRB         float64
Player_TRB         float64
Player_AST         float64
Player_STL         float64
Player_BLK         float64
Player_TOV         float64
Player_PF          float64
Player_GmSc        float64
Player_BPM         float64
Home                  bool
Player_Pos          object
Player_PTSSTDEV    float64
dtype: object


,Player,Player_PTS,Date,Team,Opp,Player_MP,Player_FGM,Player_FGA,Player_FG%,Player_2PM,...,Player_AST,Player_STL,Player_BLK,Player_TOV,Player_PF,Player_GmSc,Player_BPM,Home,Player_Pos,Player_PTSSTDEV
16134,James Harden,21.406250,2023-01-30,PHI,ORL,36.781250,6.562500,14.593750,0.456781,3.750000,...,11.156250,1.250000,0.562500,3.500000,2.187500,20.653125,5.678125,True,G,6.598433
16135,Franz Wagner,19.918367,2023-01-30,ORL,PHI,33.061224,7.224490,14.673469,0.496939,5.530612,...,3.510204,0.877551,0.204082,2.040816,2.163265,14.367347,0.253061,False,G,5.904930
16136,Matisse Thybulle,2.522727,2023-01-30,PHI,ORL,12.113636,1.000000,2.340909,0.405719,0.590909,...,0.454545,0.886364,0.318182,0.159091,1.363636,2.493182,-1.586364,True,G,3.267092
16137,Bradley Beal,22.034483,2023-01-30,WAS,SAS,33.068966,8.275862,16.206897,0.527069,6.793103,...,5.137931,0.862069,0.586207,2.689655,1.793103,16.517241,2.244828,False,G,7.277572
16138,Jock Landale,6.904762,2023-01-30,PHO,TOR,14.595238,2.571429,4.952381,0.517775,2.166667,...,0.928571,0.238095,0.523810,0.952381,1.904762,5.778571,-0.847619,True,C,4.647630


### Team Game Data

In [10]:
#### Load Data

wks_name = "TeamData"
team_df = g2d.download(gfile=spreadsheet_key, wks_name=wks_name, col_names=True, row_names=True, credentials=credentials)
team_df = team_df.reset_index()
print(team_df.shape)

(1526, 35)


In [11]:
#### Cleaning

# convert Home_Away into Boolean (Home=TRUE)
team_df['Home'] = team_df['Home_Away'] != '@'

# rename columns
team_df = team_df.rename(columns={"MP": "Team_MP",
                                     "FG": "Team_FGM",
                                     "FGA": "Team_FGA",
                                     "FG%": "Team_FG%",
                                     "2P": "Team_2PM",
                                     "2PA": "Team_2PA",
                                     "2P%": "Team_2P%",
                                     "3P": "Team_3PM",
                                     "3PA": "Team_3PA",
                                     "3P%": "Team_3P%",
                                     "FT": "Team_FTM",
                                     "FTA": "Team_FTA",
                                     "FT%": "Team_FT%",
                                     "PTS.1": "Team_PTS",
                                     "MP.1": "Opp_MP",
                                     "FG.1": "Opp_FGM",
                                     "FGA.1": "Opp_FGA",
                                     "FG%.1": "Opp_FG%",
                                     "2P.1": "Opp_2PM",
                                     "2PA.1": "Opp_2PA",
                                     "2P%.1": "Opp_2P%",
                                     "3P.1": "Opp_3PM",
                                     "3PA.1": "Opp_3PA",
                                     "3P%.1": "Opp_3P%",
                                     "FT.1": "Opp_FTM",
                                     "FTA.1": "Opp_FTA",
                                     "FT%.1": "Opp_FT%",
                                     "PTS.2": "Opp_PTS"})

# change numerical column types
numerical_columns = ['Team_MP', 'Team_FGM', 'Team_FGA', 'Team_FG%',
       'Team_2PM', 'Team_2PA', 'Team_2P%', 'Team_3PM', 'Team_3PA', 'Team_3P%',
       'Team_FTM', 'Team_FTA', 'Team_FT%', 'Team_PTS', 'Opp_FGM', 'Opp_FGA',
       'Opp_FG%', 'Opp_2PM', 'Opp_2PA', 'Opp_2P%', 'Opp_3PM', 'Opp_3PA',
       'Opp_3P%', 'Opp_FTM', 'Opp_FTA', 'Opp_FT%', 'Opp_PTS', ]
for col in numerical_columns:
    team_df = team_df.astype({col: 'float'})

# drop columns
columns_to_drop = ['index', 'Rk', 'Home_Away', 'PTS', 'Result']
team_df = team_df.drop(columns=columns_to_drop)

# add inverse rows
for index, row in team_df.iterrows():
    inverse_row = {'Team': row['Opp'],
                  'Date': row['Date'],
                  'Opp': row['Team'],
                  'Team_MP': row['Team_MP'],
                  'Team_FGM': row['Opp_FGM'],
                  'Team_FGA': row['Opp_FGA'],
                  'Team_FG%': row['Opp_FG%'],
                  'Team_2PM': row['Opp_2PM'],
                  'Team_2PA': row['Opp_2PA'],
                  'Team_2P%': row['Opp_2P%'],
                  'Team_3PM': row['Opp_3PM'],
                  'Team_3PA': row['Opp_3PA'],
                  'Team_3P%': row['Opp_3P%'],
                  'Team_FTM': row['Opp_FTM'],
                  'Team_FTA': row['Opp_FTA'],
                  'Team_FT%': row['Opp_FT%'],
                  'Team_PTS': row['Opp_PTS'],
                  'Opp_FGM': row['Team_FGM'],
                  'Opp_FGA': row['Team_FGA'],
                  'Opp_FG%': row['Team_FG%'],
                  'Opp_2PM': row['Team_2PM'],
                  'Opp_2PA': row['Team_2PA'],
                  'Opp_2P%': row['Team_2P%'],
                  'Opp_3PM': row['Team_3PM'],
                  'Opp_3PA': row['Team_3PA'],
                  'Opp_3P%': row['Team_3P%'],
                  'Opp_FTM': row['Team_FTM'],
                  'Opp_FTA': row['Team_FTA'],
                  'Opp_FT%': row['Team_FT%'],
                  'Opp_PTS': row['Team_PTS'],
                  'Home': not row['Home']}
    team_df = team_df.append(inverse_row, ignore_index=True)
    
# add boolean as float for team winning game
team_df['Win'] = (team_df['Team_PTS'] > team_df['Opp_PTS']).astype(float)

# drop repeated rows
team_df = team_df.drop_duplicates()

In [12]:
#### Manipulation - keep track of average stats coming into game

stats = ['Team_MP', 'Team_FGM', 'Team_FGA', 'Team_FG%',
       'Team_2PM', 'Team_2PA', 'Team_2P%', 'Team_3PM', 'Team_3PA', 'Team_3P%',
       'Team_FTM', 'Team_FTA', 'Team_FT%', 'Team_PTS', 'Opp_FGM', 'Opp_FGA',
       'Opp_FG%', 'Opp_2PM', 'Opp_2PA', 'Opp_2P%', 'Opp_3PM', 'Opp_3PA',
       'Opp_3P%', 'Opp_FTM', 'Opp_FTA', 'Opp_FT%', 'Opp_PTS', 'Win']

teams = team_df['Team'].unique()

# sort rows from beginning to end of season
team_df = team_df.sort_values(by=['Date'], ignore_index=True)

# data structure to hold all data
team_dict = {team:{stat:[] for stat in stats} for team in teams}

for index,row in team_df.iterrows():
    team = row['Team']
    for stat in stats:
        temp = row[stat]
        team_df.at[index,stat] = statistics.mean(team_dict[team][stat]) if len(team_dict[team][stat]) > 0 else 0
        if not pd.isna(temp): team_dict[team][stat].append(temp)  

In [13]:
#### Summary

print(team_df.shape)
print(team_df.dtypes)
team_df[-5:]

(1526, 32)
Team         object
Date         object
Opp          object
Team_MP     float64
Team_FGM    float64
Team_FGA    float64
Team_FG%    float64
Team_2PM    float64
Team_2PA    float64
Team_2P%    float64
Team_3PM    float64
Team_3PA    float64
Team_3P%    float64
Team_FTM    float64
Team_FTA    float64
Team_FT%    float64
Team_PTS    float64
Opp_FGM     float64
Opp_FGA     float64
Opp_FG%     float64
Opp_2PM     float64
Opp_2PA     float64
Opp_2P%     float64
Opp_3PM     float64
Opp_3PA     float64
Opp_3P%     float64
Opp_FTM     float64
Opp_FTA     float64
Opp_FT%     float64
Opp_PTS     float64
Home           bool
Win         float64
dtype: object


,Team,Date,Opp,Team_MP,Team_FGM,Team_FGA,Team_FG%,Team_2PM,Team_2PA,Team_2P%,...,Opp_2P%,Opp_3PM,Opp_3PA,Opp_3P%,Opp_FTM,Opp_FTA,Opp_FT%,Opp_PTS,Home,Win
1521,PHO,2023-01-30,TOR,241.960784,41.588235,89.921569,0.462549,28.705882,56.686275,0.506784,...,0.529922,11.254902,31.117647,0.358373,19.803922,25.176471,0.788137,111.568627,True,0.509804
1522,DAL,2023-01-30,DET,243.431373,39.078431,83.058824,0.471510,24.352941,42.431373,0.571373,...,0.553314,11.196078,31.333333,0.357765,19.607843,25.098039,0.780882,112.098039,True,0.509804
1523,MIN,2023-01-30,SAC,240.480769,42.500000,86.269231,0.494808,30.769231,53.403846,0.580481,...,0.533019,13.096154,35.557692,0.371519,19.442308,25.884615,0.754615,114.961538,True,0.519231
1524,ORL,2023-01-30,PHI,241.000000,40.060000,84.600000,0.475140,29.100000,53.520000,0.546160,...,0.571060,13.160000,37.840000,0.344680,18.200000,23.300000,0.782440,114.640000,False,0.380000
1525,SAC,2023-01-30,MIN,240.520833,43.083333,87.270833,0.494146,29.500000,50.541667,0.583417,...,0.560937,11.520833,31.854167,0.363937,18.145833,22.729167,0.793813,116.625000,False,0.562500


### Combine Data

In [14]:
og_player_df = player_df
og_team_df = team_df

In [15]:
player_df = og_player_df
team_df = og_team_df

# drop columns from team DataFrame
columns_to_drop = ['Opp_FGM', 'Opp_FGA', 'Opp_FG%', 'Opp_2PM', 'Opp_2PA', 'Opp_2P%', 'Opp_3PM', 'Opp_3PA', 
                   'Opp_3P%', 'Opp_FTM', 'Opp_FTA', 'Opp_FT%', 'Opp_PTS']
team_df = team_df.drop(columns=columns_to_drop)

# combine player and team data into DataFrame
on = ['Date', 'Team', 'Opp', 'Home']
df = pd.merge(player_df,team_df, on=on)

# add opponent average stats to DataFrame (x is the Player's Team, y is that team's Opp)
df = pd.merge(df, team_df, 
         left_on = ['Team', 'Date'], right_on = ['Opp', 'Date'], 
         how = 'left')

# fix Team/Home stuff
df = df.rename(columns={"Team_x": "Team", "Opp_x": "Opp", 'Home_x': 'Home', 
                        'Win_x': 'Win%_x', 'Win_y': 'Win%_y'})
df = df.drop(columns=['Team_y', 'Opp_y', 'Home_y'])

# add line data to DataFrame
on = ['Player', 'Date']
df = pd.merge(df,betting_df, on=on)

print(df.columns)
df[-5:]

Index(['Player', 'Player_PTS', 'Date', 'Team', 'Opp', 'Player_MP',
       'Player_FGM', 'Player_FGA', 'Player_FG%', 'Player_2PM', 'Player_2PA',
       'Player_2P%', 'Player_3PM', 'Player_3PA', 'Player_3P%', 'Player_FTM',
       'Player_FTA', 'Player_FT%', 'Player_TS%', 'Player_ORB', 'Player_DRB',
       'Player_TRB', 'Player_AST', 'Player_STL', 'Player_BLK', 'Player_TOV',
       'Player_PF', 'Player_GmSc', 'Player_BPM', 'Home', 'Player_Pos',
       'Player_PTSSTDEV', 'Team_MP_x', 'Team_FGM_x', 'Team_FGA_x',
       'Team_FG%_x', 'Team_2PM_x', 'Team_2PA_x', 'Team_2P%_x', 'Team_3PM_x',
       'Team_3PA_x', 'Team_3P%_x', 'Team_FTM_x', 'Team_FTA_x', 'Team_FT%_x',
       'Team_PTS_x', 'Win%_x', 'Team_MP_y', 'Team_FGM_y', 'Team_FGA_y',
       'Team_FG%_y', 'Team_2PM_y', 'Team_2PA_y', 'Team_2P%_y', 'Team_3PM_y',
       'Team_3PA_y', 'Team_3P%_y', 'Team_FTM_y', 'Team_FTA_y', 'Team_FT%_y',
       'Team_PTS_y', 'Win%_y', 'Line'],
      dtype='object')


,Player,Player_PTS,Date,Team,Opp,Player_MP,Player_FGM,Player_FGA,Player_FG%,Player_2PM,...,Team_2P%_y,Team_3PM_y,Team_3PA_y,Team_3P%_y,Team_FTM_y,Team_FTA_y,Team_FT%_y,Team_PTS_y,Win%_y,Line
6640,Cole Anthony,12.218750,2023-01-30,ORL,PHI,25.937500,4.250000,9.687500,0.422187,3.093750,...,0.547187,12.875,33.458333,0.38125,20.0,24.208333,0.824771,114.791667,0.666667,10.5
6641,Gary Harris,9.090909,2023-01-30,ORL,PHI,23.772727,3.272727,6.409091,0.493682,1.545455,...,0.547187,12.875,33.458333,0.38125,20.0,24.208333,0.824771,114.791667,0.666667,7.5
6642,Wendell Carter Jr.,15.258065,2023-01-30,ORL,PHI,30.258065,5.612903,10.677419,0.525290,4.387097,...,0.547187,12.875,33.458333,0.38125,20.0,24.208333,0.824771,114.791667,0.666667,13.5
6643,Markelle Fultz,12.068966,2023-01-30,ORL,PHI,27.896552,5.034483,10.068966,0.490414,4.586207,...,0.547187,12.875,33.458333,0.38125,20.0,24.208333,0.824771,114.791667,0.666667,11.5
6644,Franz Wagner,19.918367,2023-01-30,ORL,PHI,33.061224,7.224490,14.673469,0.496939,5.530612,...,0.547187,12.875,33.458333,0.38125,20.0,24.208333,0.824771,114.791667,0.666667,17.5


In [16]:
# write combined data to Google Sheets

wks_name = 'CombinedData'
d2g.upload(df, spreadsheet_key, wks_name, credentials=credentials, row_names=True)
print(df.shape)

(6645, 63)


## Predictions

### Functions

In [18]:
# returns only selected features of X

def select_features(X, y):
    
    feature_selection_pipeline = Pipeline([
        ('scaler',StandardScaler()),
        ('model',Lasso())
    ])
    
    search = GridSearchCV(feature_selection_pipeline,
                          {'model__alpha':np.arange(0.1,10,0.1)},
                          cv = 5, scoring="neg_mean_squared_error",verbose=0
                         )
    search.fit(X, y)

    coefficients = search.best_estimator_.named_steps['model'].coef_
    importance = np.abs(coefficients)
    selected_features = np.array(X.columns)[importance > 0]
    print("Selected:", selected_features)

    unselected_features = np.array(X.columns)[importance == 0]
    print("Unselected:", unselected_features)
    print()

    return X[selected_features]

In [19]:
# return probability given line, estimate, and uncertainty

def regression_probability(line, estimate, uncertainty):
    
    z_score = abs(line-estimate) / uncertainty
    return (1-norm.sf(z_score))*100

### Data Preparation

In [20]:
og_df = df

In [21]:
df = og_df

#### Prepare Training/Testing Data

# remove data before a given date
df = df[df['Date'] > '2022-11-18']

# drop na values

# shuffle DataFrame
df = df.sample(frac=1).reset_index(drop=True)

# X y split(s)
input_cols = ['Player_MP',
       'Player_FGM', 'Player_FGA', 'Player_FG%', 'Player_2PM', 'Player_2PA',
       'Player_2P%', 'Player_3PM', 'Player_3PA', 'Player_3P%', 'Player_FTM',
       'Player_FTA', 'Player_FT%', 'Player_TS%', 'Player_ORB', 'Player_DRB',
       'Player_TRB', 'Player_AST', 'Player_STL', 'Player_BLK', 'Player_TOV',
       'Player_PF', 'Player_GmSc', 'Player_BPM', 'Home', 'Player_Pos',
       'Player_PTSSTDEV', 'Team_MP_x', 'Team_FGM_x', 'Team_FGA_x',
       'Team_FG%_x', 'Team_2PM_x', 'Team_2PA_x', 'Team_2P%_x', 'Team_3PM_x',
       'Team_3PA_x', 'Team_3P%_x', 'Team_FTM_x', 'Team_FTA_x', 'Team_FT%_x',
       'Team_PTS_x', 'Win%_x', 'Team_MP_y', 'Team_FGM_y', 'Team_FGA_y',
       'Team_FG%_y', 'Team_2PM_y', 'Team_2PA_y', 'Team_2P%_y', 'Team_3PM_y',
       'Team_3PA_y', 'Team_3P%_y', 'Team_FTM_y', 'Team_FTA_y', 'Team_FT%_y',
       'Team_PTS_y', 'Win%_y']

X = df[input_cols]
players = df['Player']
dates = df['Date']
lines = df['Line']
pts = df['Player_PTS']
bets = pts > lines # TRUE=OVER

# OneHotEncoder: Home 
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(df[['Home']]).toarray())
X = X.join(enc_df)
X = X.drop(columns=['Home'])
X = X.rename(columns={0: "Away", 1: "Home"})

# OneHotEncoder: Player_Pos
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(df[['Player_Pos']]).toarray())
X = X.join(enc_df)
X = X.drop(columns=['Player_Pos'])
X = X.rename(columns={0: "C", 1: "F", 2: "G"})

# Polynomial Features

# Scale Data 
X = pd.DataFrame(StandardScaler().fit_transform(X), columns=X.columns)

X

,Player_MP,Player_FGM,Player_FGA,Player_FG%,Player_2PM,Player_2PA,Player_2P%,Player_3PM,Player_3PA,Player_3P%,...,Team_FTM_y,Team_FTA_y,Team_FT%_y,Team_PTS_y,Win%_y,Away,Home,C,F,G
0,0.281276,0.592393,0.516606,0.077876,0.931586,1.016261,-0.193599,-0.544745,-0.471853,0.186124,...,-0.804612,-0.316121,-0.999177,1.358820,1.663241,1.017644,-1.017644,-0.372326,-0.744316,0.957408
1,-0.838293,-0.922411,-1.158241,1.397873,-0.434767,-0.629711,1.075112,-1.204616,-1.273673,-0.252347,...,0.158648,-0.430887,1.106991,0.453397,-0.301676,1.017644,-1.017644,-0.372326,1.343515,-1.044486
2,1.404086,0.605016,0.643340,-0.119244,0.564804,0.624630,0.056665,0.224996,0.318767,0.081957,...,0.904665,1.047300,-0.305397,0.274887,0.289083,-0.982661,0.982661,-0.372326,1.343515,-1.044486
3,0.224133,-0.725819,-0.908998,0.870427,-1.032646,-1.195118,1.245187,0.447637,-0.008402,1.746988,...,1.055603,0.720691,0.714786,-1.256237,-1.286275,-0.982661,0.982661,2.685820,-0.744316,-1.044486
4,0.239927,0.498401,0.962097,-0.839681,0.201577,0.665409,-1.335916,0.718257,0.856696,0.115220,...,-0.485069,-0.777496,0.496162,0.624631,-0.933178,1.017644,-1.017644,-0.372326,1.343515,-1.044486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4455,0.281169,0.614874,0.530531,0.091531,0.968986,1.048656,-0.182088,-0.569554,-0.491645,0.114464,...,-0.500624,-1.104870,1.237103,0.724204,-0.006297,-0.982661,0.982661,-0.372326,-0.744316,0.957408
4456,0.031087,0.187528,-0.319523,1.103055,0.470334,0.011878,1.010212,-0.526974,-0.613824,0.357700,...,-1.485878,-1.026607,-1.209091,-2.271649,0.600009,1.017644,-1.017644,-0.372326,1.343515,-1.044486
4457,0.355755,-0.600129,-0.743086,0.737089,-0.868770,-1.066165,1.242247,0.400326,0.119248,1.376444,...,0.486114,1.239993,-1.526541,-1.352490,0.473695,-0.982661,0.982661,2.685820,-0.744316,-1.044486
4458,1.053707,1.179913,1.228864,-0.159805,0.868066,0.832574,0.117486,0.910520,1.118756,0.354163,...,-0.463034,0.199103,-1.122359,-0.282279,1.060352,1.017644,-1.017644,-0.372326,1.343515,-1.044486


In [22]:
# save X DataFrame
X_saved = X

### Regression

In [23]:
X = X_saved

# Feature Selection
X = select_features(X, pts)

# sample size
n = len(X)

# model selection + training
regression_model = BayesianRidge(n_iter=1000)
regression_model.fit(X, pts)

# predictions
model_output = regression_model.predict(X, return_std=True)
estimates = model_output[0]
predictions = estimates > lines
uncertainties = model_output[1]
probabilities = regression_probability(lines, estimates, uncertainties)

regression_df = pd.DataFrame({'Player': players, 
                             'Points': pts, 
                             'Estimates': estimates,
                              'Line': lines,
                              'Actual': bets,
                            'RegressionPrediction': predictions,
                             'RegressionUncertainty': uncertainties,
                             'RegressionProbability': probabilities})

# results
accuracy = statistics.mean(regression_df['Actual'] == regression_df['RegressionPrediction'])
mae = metrics.mean_absolute_error(pts, estimates)

print("Accuracy:", accuracy)
print("MAE:", mae)
print("Average Uncertainty:", statistics.mean(uncertainties))
regression_df

Selected: ['Player_FGM' 'Player_FGA' 'Player_3PM' 'Player_FTM' 'Player_GmSc']
Unselected: ['Player_MP' 'Player_FG%' 'Player_2PM' 'Player_2PA' 'Player_2P%'
 'Player_3PA' 'Player_3P%' 'Player_FTA' 'Player_FT%' 'Player_TS%'
 'Player_ORB' 'Player_DRB' 'Player_TRB' 'Player_AST' 'Player_STL'
 'Player_BLK' 'Player_TOV' 'Player_PF' 'Player_BPM' 'Player_PTSSTDEV'
 'Team_MP_x' 'Team_FGM_x' 'Team_FGA_x' 'Team_FG%_x' 'Team_2PM_x'
 'Team_2PA_x' 'Team_2P%_x' 'Team_3PM_x' 'Team_3PA_x' 'Team_3P%_x'
 'Team_FTM_x' 'Team_FTA_x' 'Team_FT%_x' 'Team_PTS_x' 'Win%_x' 'Team_MP_y'
 'Team_FGM_y' 'Team_FGA_y' 'Team_FG%_y' 'Team_2PM_y' 'Team_2PA_y'
 'Team_2P%_y' 'Team_3PM_y' 'Team_3PA_y' 'Team_3P%_y' 'Team_FTM_y'
 'Team_FTA_y' 'Team_FT%_y' 'Team_PTS_y' 'Win%_y' 'Away' 'Home' 'C' 'F' 'G']

Accuracy: 0.997982062780269
MAE: 4.411477660500221e-13
Average Uncertainty: 2.11999286704187e-05


,Player,Points,Estimates,Line,Actual,RegressionPrediction,RegressionUncertainty,RegressionProbability
0,Josh Giddey,16.000000,16.000000,17.5,False,False,0.000021,100.0
1,Jarred Vanderbilt,8.282051,8.282051,8.5,False,False,0.000021,100.0
2,OG Anunoby,18.470588,18.470588,15.5,True,True,0.000021,100.0
3,Al Horford,10.000000,10.000000,11.5,False,False,0.000021,100.0
4,Dillon Brooks,17.720000,17.720000,18.5,False,False,0.000021,100.0
...,...,...,...,...,...,...,...,...
4455,Josh Giddey,16.047619,16.047619,16.5,False,False,0.000021,100.0
4456,Aaron Gordon,15.733333,15.733333,15.5,True,True,0.000021,100.0
4457,Al Horford,10.642857,10.642857,10.5,True,True,0.000021,100.0
4458,Kyle Kuzma,21.351351,21.351351,20.5,True,True,0.000021,100.0


### Classification

In [24]:
X = X_saved

# Feature Selection ??

# sample size
n = len(X)

# model selection + training
classification_model = GaussianProcessClassifier()
# classification_model = MLPClassifier(max_iter=1000)
classification_model.fit(X, bets)

# predictions
predictions = classification_model.predict(X)
probabilities = [round(max(vals)*100, 2) for vals in classification_model.predict_proba(X)]

classification_df = pd.DataFrame({'Player': players, 
                             'Points': pts,
                              'Line': lines,
                              'Actual': bets,
                            'ClassificationPrediction': predictions,
                             'ClassificationProbability': probabilities})

# results
accuracy = statistics.mean(classification_df['Actual'] == classification_df['ClassificationPrediction'])

print("Accuracy:", accuracy)
classification_df

Accuracy: 1


,Player,Points,Line,Actual,ClassificationPrediction,ClassificationProbability
0,Josh Giddey,16.000000,17.5,False,False,58.53
1,Jarred Vanderbilt,8.282051,8.5,False,False,58.48
2,OG Anunoby,18.470588,15.5,True,True,58.81
3,Al Horford,10.000000,11.5,False,False,52.07
4,Dillon Brooks,17.720000,18.5,False,False,58.47
...,...,...,...,...,...,...
4455,Josh Giddey,16.047619,16.5,False,False,58.48
4456,Aaron Gordon,15.733333,15.5,True,True,58.47
4457,Al Horford,10.642857,10.5,True,True,58.47
4458,Kyle Kuzma,21.351351,20.5,True,True,52.10


### Testing

In [25]:
# combine Regression and Classification DataFrames
on = ['Player', 'Points', 'Line', 'Actual']
prediction_df = pd.merge(regression_df,classification_df, on=on)
prediction_df

# only select bets where predictions match
prediction_df = prediction_df[prediction_df['RegressionPrediction'] == prediction_df['ClassificationPrediction']]

# minimum probability thresholds for bets
prediction_df = prediction_df[prediction_df['RegressionProbability'] > 50]
prediction_df = prediction_df[prediction_df['ClassificationProbability'] > 50]

# results
accuracy = statistics.mean(prediction_df['Actual'] == prediction_df['ClassificationPrediction'])
print("Accuracy: " + str(round(accuracy*100,2)) + "%")
prediction_df

Accuracy: 100%


,Player,Points,Estimates,Line,Actual,RegressionPrediction,RegressionUncertainty,RegressionProbability,ClassificationPrediction,ClassificationProbability
0,Josh Giddey,16.000000,16.000000,17.5,False,False,0.000021,100.0,False,58.53
1,Jarred Vanderbilt,8.282051,8.282051,8.5,False,False,0.000021,100.0,False,58.48
2,OG Anunoby,18.470588,18.470588,15.5,True,True,0.000021,100.0,True,58.81
3,Al Horford,10.000000,10.000000,11.5,False,False,0.000021,100.0,False,52.07
4,Dillon Brooks,17.720000,17.720000,18.5,False,False,0.000021,100.0,False,58.47
...,...,...,...,...,...,...,...,...,...,...
4501,Josh Giddey,16.047619,16.047619,16.5,False,False,0.000021,100.0,False,58.48
4502,Aaron Gordon,15.733333,15.733333,15.5,True,True,0.000021,100.0,True,58.47
4503,Al Horford,10.642857,10.642857,10.5,True,True,0.000021,100.0,True,58.47
4504,Kyle Kuzma,21.351351,21.351351,20.5,True,True,0.000021,100.0,True,52.10
